In [6]:
####################################Imports and Initial Feed Rip###################################################
import feedparser as fp, pandas as pd, datetime, time, string, requests as r
from bs4 import BeautifulSoup 
from functools import reduce


rss_feed = 'http://feeds.arstechnica.com/arstechnica/index?format=xml'
company_info_df = pd.read_csv('Target_Company.csv')
company_name = company_info_df.iloc[0]['Company Name']


#Get the News Feed and Slice it to only include the news items
feed = fp.parse(rss_feed)
feed_items = feed['items']
print(feed_items[0])
###################################################################################################################

{'published': 'Sat, 09 Apr 2016 18:15:19 +0000', 'title_detail': {'language': None, 'type': 'text/plain', 'value': 'Mounting data suggest antibacterial soaps do more harm than good', 'base': 'http://feeds.arstechnica.com/arstechnica/index?format=xml'}, 'summary_detail': {'language': None, 'type': 'text/html', 'value': 'Few pros, but cons include upped risk of infection, microbiome changes, drug resistance.', 'base': 'http://feeds.arstechnica.com/arstechnica/index?format=xml'}, 'author_detail': {'name': 'Beth Mole'}, 'authors': [{'name': 'Beth Mole'}], 'tags': [{'label': None, 'scheme': None, 'term': 'Scientific Method'}, {'label': None, 'scheme': None, 'term': 'antibacterial soap'}, {'label': None, 'scheme': None, 'term': 'antibiotic resistance'}, {'label': None, 'scheme': None, 'term': 'triclosan'}], 'id': 'http://arstechnica.com/?p=862109', 'author': 'Beth Mole', 'link': 'http://arstechnica.com/science/2016/04/mounting-data-suggest-antibacterial-soaps-do-more-harm-than-good/', 'title

In [12]:
#################################Create the Cleaned News Items DataFrame###########################################

#Import the raw data into a DataFrame
raw_news_dataframe = pd.DataFrame(feed_items)

#Clean the DataFrame
def dataframe_cleaner(dataframe): 
    being_cleaned_df = pd.DataFrame(raw_news_dataframe[['author','id','title','content']])
    
    #for i in range(0,len(being_cleaned_df['content'])):
        #soup = BeautifulSoup(being_cleaned_df['content'][i],'html.parser')
        #print(soup)
        #being_cleaned_df['content'][i] = soup.get_text()
    
    return being_cleaned_df
    
#Create the Full Dataframe    
full_df = dataframe_cleaner(raw_news_dataframe)

#Create the filtered dataframe, containing only rows that have company_name in the title
company_in_title_list = list(map(lambda x: company_name in x,full_df['title']))
news_df = full_df[company_in_title_list].reset_index(drop=True)
news_df.head()

###################################################################################################################

TypeError: expected string or bytes-like object

In [4]:
#Load in the Sentiment DataFrame
sentiment_df = pd.read_csv('../Sentiment_Dictionary/sentiment_DataFrame.csv')
sentiment_df = sentiment_df.ix[:,'type':]
sentiment_df.head(1)

,type,word,position,stemmed,polarity
0,0.5,abandoned,adj,n,-1


In [5]:
##########################Perform Sentiment Analysis on the Title and Body of the Articles#########################
def title_analysis(title):
    tokenized_title = dict.fromkeys(title.split(' '))                          
                                
    for key in tokenized_title:
        tokenized_title[key] = [sentiment_df['type'][i] * sentiment_df['polarity'][i]
                               for i in range(0,len(sentiment_df.index)) 
                               if key.lower() == sentiment_df['word'][i]]
    
    total_sentiment = sum([item for sublist in list(tokenized_title.values()) for item in sublist])
    normalized_sentiment = total_sentiment / len(tokenized_title)
    
    return normalized_sentiment

def content_analysis(content):
    tokenized_content = dict.fromkeys(content.split(' '))
    
    for key in tokenized_content:
        tokenized_content[key] = [sentiment_df['type'][i] * sentiment_df['polarity'][i]
                               for i in range(0,len(sentiment_df.index)) 
                               if key.lower() == sentiment_df['word'][i]]
    
    total_sentiment = sum([item for sublist in list(tokenized_content.values()) for item in sublist])
    normalized_sentiment = total_sentiment / len(tokenized_content)
    return normalized_sentiment

def sentiment_analysis(dataframe):
    analyzing_df = dataframe
    analyzing_df['normalized_title_sentiment'] = list(map(lambda x: title_analysis(x),analyzing_df['title']))
    analyzing_df['normalized_content_sentiment'] = list(map(lambda x: content_analysis(x),analyzing_df['content']))
    return analyzing_df

start_time = time.time()
analyzed_articles_df = sentiment_analysis(news_df)
elapsed_time = time.time() - start_time
print('Elapsed Time: ' + str(elapsed_time))
analyzed_articles_df.head(1)
analyzed_articles_df.to_csv('ArsTechnicaSentiment.csv')
###################################################################################################################

Elapsed Time: 1.54046630859375
